In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")
sns.set_palette("Set2")

# Project
## Apartment Recommendation System

In [32]:
df=pd.read_csv(r"c:\Users\user\Desktop\Kaggle Datasets\Housing_Price_Data.csv")

# Data Info

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [34]:
df.shape

(545, 13)

In [35]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


# Data Cleaning & Handling

In [36]:
df.isna().sum()
df.duplicated().sum()

np.int64(0)

# Reccomendation System For perfect apartment 

In [37]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [38]:
cat_cols=df.select_dtypes(include="object").columns
df_encoded=df.copy()
le=LabelEncoder()
for col in cat_cols:
    df_encoded[col]=le.fit_transform(df_encoded[col])
X=df_encoded.drop('price',axis=1)
y=df_encoded["price"]
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [39]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
model=XGBRegressor()
model.fit(X_train,y_train)

,objective,'reg:squarederror'
,base_score,0.5
,booster,'gbtree'
,callbacks,None
,colsample_bylevel,1
,colsample_bynode,1
,colsample_bytree,1
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
,gamma,0


In [40]:
# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("R² Score:", round(r2_score(y_test, y_pred), 3))

R² Score: 0.626


# Apartment Recommendation System

In [41]:
df["price"]

0      13300000
1      12250000
2      12250000
3      12215000
4      11410000
         ...   
540     1820000
541     1767150
542     1750000
543     1750000
544     1750000
Name: price, Length: 545, dtype: int64

In [44]:
for col in ["airconditioning", "guestroom", "prefarea"]:
    df[col] = df[col].str.strip().str.lower()


In [49]:
# Filter based on preferences
user_pref = df[
    (df["airconditioning"] == "yes") &
    (df["guestroom"] == "no") &
    (df["prefarea"] == "no") &
    (df["price"] <= 12250000)
]

# Sort by price or area
recommended = user_pref.sort_values(by="area", ascending=False).head(5)
print("Top 5 Recommended Apartments:")
recommended[["area", "bedrooms", "bathrooms", "price", "furnishingstatus"]]

Top 5 Recommended Apartments:


,area,bedrooms,bathrooms,price,furnishingstatus
125,15600,3,1,5943000,semi-furnished
222,9166,2,1,4760000,semi-furnished
37,9000,4,2,7980000,furnished
1,8960,4,4,12250000,furnished
61,8880,2,1,7070000,semi-furnished
